In [ ]:
import geopandas as gpd
import requests
import json
from pandas import json_normalize
from shapely.geometry import Polygon
from geojson import Feature, FeatureCollection

In [ ]:
def up_load_shp(path):
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [ ]:
shapefile_path=r'C:\Users\dpere\Documents\Bandicut\geopandas\polygonFromGidon\polygon.shp'
polygonFromGidon=up_load_shp(shapefile_path)

In [ ]:
minx = polygonFromGidon.bounds.minx[0]
miny = polygonFromGidon.bounds.miny[0]
maxx = polygonFromGidon.bounds.maxx[0]
maxy = polygonFromGidon.bounds.maxy[0]

In [ ]:
url = r'https://ags.iplan.gov.il/arcgisiplan/rest/services/PlanningPublic/Xplan/MapServer/1/query?f=json&where=pl_area_dunam%20<%3D15&returnGeometry=true&geometry=%7B%22xmin%22%3A{}%2C%22ymin%22%3A{}%2C%22xmax%22%3A{}%2C%22ymax%22%3A{}%2C%22spatialReference%22%3A%7B%22wkid%22%3A2039%7D%7D&geometryType=esriGeometryEnvelope&inSR=2039&outFields=pl_number%2Cpl_name%2Cpl_url%2Cquantity_delta_120%2Cstation_desc%2Cplan_county_name&orderByFields=pl_number&outSR=2039'.format(minx, miny, maxx, maxy)
response = requests.get(url)
response_json = response.json()

In [ ]:
data = json.dumps(response_json)
dict= json.loads(data)
df = json_normalize(dict['features'])

In [ ]:
features = []

for feature in dict['features']:
    interior_rings = feature['geometry']['rings'][1:]
    exterior_ring=feature['geometry']['rings'][0]
    polygon = Polygon(exterior_ring, holes=interior_rings)
    if polygonFromGidon.contains(polygon.centroid).bool():
        feature = Feature(geometry=polygon, properties=feature['attributes'])
        features.append(feature)
        
crs = {
    "type": "name",
    "properties": {
        "name": "EPSG:2039"
    }
}

feature_collection = FeatureCollection(features)

gdf = gpd.GeoDataFrame.from_features(feature_collection, crs='EPSG:2039')
gdf.to_file('my_polygon/my_polygon.shp', encoding='utf-8')
gdf
